In [1]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob

from utils import load_paths

In [2]:

def all_combine_nofilter(
    files_by_trait,
    cols_keep,
    output_folder="/Users/adama/Downloads/Tchandjieu_Lab/Adama_bnmf_handover/files/combined_gwas"
):
    os.makedirs(output_folder, exist_ok=True)
    
    df_dict = {}
    combined_file = os.path.join(output_folder, "all_cardiac_traits_raw.csv")
    first_write = True  # for appending to CSV safely

    with open(combined_file, 'w') as f:  # Clear any old file early
        pass

    for trait, file_list in files_by_trait.items():
        trait_output_path = os.path.join(output_folder, f"{trait}_prepruned.csv")
        first_trait_write = True

        for filepath in file_list:
            if not filepath.endswith(('.regenie', '.regenie.gz')):
                print(f"Skipping invalid file: {filepath}")
                continue

            try:
                df = pd.read_csv(filepath, delim_whitespace=True, usecols=lambda c: c in cols_keep)
                df = df[df['ID'].str.startswith('rs')]

                label = '_'.join(os.path.basename(filepath).replace('.regenie.gz', '').replace('.regenie', '').split('_')[-3:])
                df['trait'] = label

                df.to_csv(trait_output_path, mode='a', header=first_trait_write, index=False)
                df.to_csv(combined_file, mode='a', header=first_write, index=False)

                first_trait_write = False
                first_write = False

            except Exception as e:
                print(f"❌ Error processing {filepath}: {e}")

    print("✅ All traits combined and saved (in chunks).")
    return None, None  # Omit storing large dataframes in memory

In [4]:
main_folder = "/Users/adama/Gladstone Dropbox/Adama Saccoh/March_18_ukbrap_wynton"
traits = ["aortic_area", "atrial_volume","ventrical_volume" ,"wall_thickness"]

files_use=load_paths(traits,main_folder)

df_dict, all_traits_df=all_combine_nofilter(files_use[1],
    cols_keep=['ID', 'LOG10P','CHROM', 'GENPOS','ALLELE0','ALLELE1','A1FREQ','BETA','SE'],
    output_folder="/Users/adama/Downloads/Tchandjieu_Lab/Adama_bnmf_handover/files/gwas_sumstat/gwas_unfltered")

Skipping invalid file: /Users/adama/Gladstone Dropbox/Adama Saccoh/March_18_ukbrap_wynton/aortic_area/step_2_aortic_area/merged/Icon
✅ All traits combined and saved (in chunks).


In [ ]:
data=pd.read_csv('/Users/adama/Downloads/Tchandjieu_Lab/NMF_python/raw_data/all_cardiac_traits_raw.csv')

data['full_id_10']=data["CHROM"].astype(str) + ":" + \
                                       data["GENPOS"].astype(str) + ":" + \
                                       data["ALLELE1"] + ":" + data["ALLELE0"]


data['full_id_01']=data["CHROM"].astype(str) + ":" + \
                                       data["GENPOS"].astype(str) + ":" + \
                                       data["ALLELE0"] + ":" + data["ALLELE1"]